In [19]:
import pandas as pd
import numpy as np


def replenish_ip(ip):
    '''
    补全ip中的 :: 为0000
    补全ip中前导0的省略
    '''
    org_length = 8
    splited_ = ip.split(':')
    if '' in splited_:
        length = len(splited_) - 1
    else:
        length = len(splited_)
    d = org_length - length
    if ip.endswith('::'):
        ip = ip.replace('::', ':0000'*d)
    else:
        ip = ip.replace('::', ':0000'*d + ':')
    _temp = ip.split(':')
    # print(temp)
    for index, i in enumerate(_temp):
        l = len(i)
        if l < 4:
            d = 4 - l
            _temp[index] = '0'*d + i

    return ':'.join(_temp)


def hex_to_bin(ip):
    # 排除奇奇怪怪的东西 "2.at.pool.ntp.org", '[2a0a' 这样的东西
    if ':' not in ip or '[' in ip:
        return 'error_ip' # 记住在进行apply完之后要检查一遍 'error_ip'
    ip = replenish_ip(ip)
    ip = ip.split(':')
    memory_list = []
    for i in ip:
        temp__ = int(i, 16)
        b = bin(temp__)[2:]
        x = len(b)
        if x < 16:
            d = 16 - x
            b = '0'*d + b
        memory_list.append(b)
    final = ''.join(memory_list)
    
    return final[104:]

In [2]:
path = 'D:/ipv6_label_lzx20190904.csv'

df = pd.read_csv(path)
df

,ip,label
0,240e:e2:1c:31af:d260:8c21:5a52:4bcd,2
1,240e:e2:1c:31af:b6de:df46:5a39:8168,2
2,240e:e2:1c:31af:8a5d:fbf0:9f6a:1b38,2
3,240e:e2:1c:31af:76e1:9a29:46d3:ab00,2
4,240e:e2:1c:31af:6a9f:f0c9:755d:819b,2
5,240e:e2:1c:31af:62bb:c83:ff62:3732,2
6,240e:e2:1c:31ab:a3f:bc6a:adad:a903,2
7,240e:e2:1c:31ab:36da:b714:372c:69cf,2
8,240e:e2:1c:31ab:2be:9ec1:db01:32c0,2
9,240e:e2:1c:31ab:1288:ce96:1ac7:6694,2


In [3]:
temp = df.copy()

temp = temp[temp['label'] == 0] # 选出 DHCP

In [4]:
temp = temp['ip']
temp

260        240e:e2:1c:2f99:e6bd:4bde:b4bb:9d9f
380         240e:e2:1c:2eb4:b2e2:e522:cc0:ebe8
388         240e:e2:1c:2eb4:2be:9e8e:832e:88b0
428          240e:e2:1c:2e0b:a3f:bc0f:b0ba:b04
635        240e:e2:1c:2c20:6a9f:f02a:935f:aaaf
                          ...                 
3373667                       2001:43f8:11f0::
3373668                      2001:7fa:3:ca07::
3373669                          2001:7fa:11::
3373670                          2001:504:2c::
3373671                         2001:7f8:1:1::
Name: ip, Length: 1995755, dtype: object

In [5]:
type(temp)

pandas.core.series.Series

In [6]:
temp[temp.str.contains('4ixp.com')]

2766842    4ixp.com
Name: ip, dtype: object

In [7]:
print(temp[2766841])
print(temp[2766842])
temp.drop(index=2766842, inplace=True)# 删除异常的值
print(temp[2766843])

print('**'*30)

# print(temp[2766841])
# print(temp[2766842])
# print(temp[2766843])  应当重置索引

2001:1250:ffe0:2::1
4ixp.com
2c0f:feb0:d:1::8
************************************************************


In [18]:
%%time
temp_ = temp.apply(hex_to_bin)
temp_

Wall time: 17.1 s


260        101110111001110110011111
380        110000001110101111101000
388        001011101000100010110000
428        101110100000101100000100
635        010111111010101010101111
                     ...           
3373667                    00000000
3373668                    00000000
3373669                    00000000
3373670                    00000000
3373671                    00000000
Name: ip, Length: 1995754, dtype: object

In [22]:
temp_[temp_ == 'error_ip'] # 190w 条数据里面有 863条 异常ip

2768627    error_ip
3347362    error_ip
3347364    error_ip
3347365    error_ip
3347366    error_ip
             ...   
3348619    error_ip
3348622    error_ip
3348628    error_ip
3348648    error_ip
3348649    error_ip
Name: ip, Length: 863, dtype: object

In [23]:
temp_[temp_ == 'error_ip'].index

Int64Index([2768627, 3347362, 3347364, 3347365, 3347366, 3347368, 3347369,
            3347370, 3347371, 3347372,
            ...
            3348614, 3348615, 3348616, 3348617, 3348618, 3348619, 3348622,
            3348628, 3348648, 3348649],
           dtype='int64', length=863)

In [24]:
err_index = temp_[temp_ == 'error_ip'].index

print(temp_.shape)
temp_.drop(index=err_index, inplace=True)
print(temp_.shape)

(1995754,)
(1994891,)


In [25]:
1995754 - 1994891

863

In [26]:
temp_[3373667]

'00000000'

In [28]:
temp[3373667]

'2001:43f8:11f0::'

In [34]:
temp_[temp_.str.len() < 24]

2864       00000000
19363      00000000
20065      00000000
20077      00000000
20089      00000000
             ...   
3373667    00000000
3373668    00000000
3373669    00000000
3373670    00000000
3373671    00000000
Name: ip, Length: 109869, dtype: object

In [36]:
temp_[temp_.str.len() < 24].str.len() # 查看算法是否有异常，是不是 len < 24 的都是全 0 的 

2864       8
19363      8
20065      8
20077      8
20089      8
          ..
3373667    8
3373668    8
3373669    8
3373670    8
3373671    8
Name: ip, Length: 109869, dtype: int64

In [42]:
temp_[temp_.str.len() < 24][temp_.str.len() != 8] 
# 证明 字符串 len < 24 即 len == 8 的都是全0的字符串，算法没有错误

Series([], Name: ip, dtype: object)

In [47]:
temp_.to_csv('D:/from_104_to_128.csv', index=False)

d:\anaconda_file\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [49]:
temp_.to_pickle('D:/from_104_to_128.pk')

In [50]:
pd.read_pickle('D:/from_104_to_128.pk')

260        101110111001110110011111
380        110000001110101111101000
388        001011101000100010110000
428        101110100000101100000100
635        010111111010101010101111
                     ...           
3373667                    00000000
3373668                    00000000
3373669                    00000000
3373670                    00000000
3373671                    00000000
Name: ip, Length: 1994891, dtype: object

In [52]:
temp[260]

'240e:e2:1c:2f99:e6bd:4bde:b4bb:9d9f'